# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233535785100                   -0.50    7.0         
  2   -7.249521040562       -1.80       -1.40    1.0   5.13ms
  3   -7.251205873273       -2.77       -1.83    2.0   6.07ms
  4   -7.251291203985       -4.07       -2.28    1.0   5.08ms
  5   -7.251278382732   +   -4.89       -2.27    2.0   6.13ms
  6   -7.251335991221       -4.24       -2.89    1.0   5.25ms
  7   -7.251338693352       -5.57       -3.53    1.0   5.32ms
  8   -7.251338757467       -7.19       -3.73    2.0   6.65ms
  9   -7.251338795314       -7.42       -4.40    1.0   5.50ms
 10   -7.251338798008       -8.57       -4.58    3.0   7.62ms
 11   -7.251338798631       -9.21       -5.28    1.0   5.65ms
 12   -7.251338798689      -10.24       -5.35    3.0   7.78ms
 13   -7.251338798699      -10.99       -5.65    1.0   5.81ms
 14   -7.251338798704      -11.30       -6.49    2.0   6.83ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.1112294440833354
[ Info: Arnoldi iteration step 2: normres = 0.538288866895305
[ Info: Arnoldi iteration step 3: normres = 0.5497281062773849
[ Info: Arnoldi iteration step 4: normres = 0.23151664748630876
[ Info: Arnoldi iteration step 5: normres = 0.5196235621273637
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.95e-03, 5.78e-02, 4.68e-01, 2.18e-01, 1.79e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2806705900503025
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.49e-03, 2.13e-01, 1.17e-01, 1.21e-01, 4.26e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06604900991618669
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.79e-05, 1.16e-02, 5.26e-03, 3.16e-02, 4.98e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09925193372426334
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.29e-06